Q. If you have trained five different models on the exact same training data, and they all achieve 95% precision, is there any chance that you can combine these models to get better results? If so, how? If not, why?

A. Yes - they may have made different errors so modal results may improve the performance

Q. What is the difference between hard and soft voting classifiers?

A.
- hard uses the modal classification
- soft uses the combined mean probabilities to wieght the vote so high confidence votes count for more



Q. Is it possible to speed up training of a bagging ensemble by distributing it across multiple servers? What about pasting ensembles, boosting ensembles, random forests, or stacking ensembles?

A. 
- Bagging; yes models can be trainined in parrallel as they are independent
- pasting; yes
- boosting; no - models need to be sequentially trained
- random forests - yes
- stacking - yes models on different layers can be trained in parrallel

Q. What is the benefit of out-of-bag evaluation?

A. A 'free' out-of-sample estimate.

Q. What makes Extra-Trees more random than regular Random Forests? How can this extra randomness help? Are Extra-Trees slower or faster than regular Random Forests?

They pick random thresholds at each node. This speeds up training considerably and can perform better if random forest is overfitting.



Q. If your AdaBoost ensemble underfits the training data, what hyperparameters should you tweak and how?

A. Number of estimators as more may be required to find a good solution, and reduce regularization of the inital estimator to free up model params for more tweaking.

Q. If your Gradient Boosting ensemble overfits the training set, should you increase or decrease the learning rate?

A. Decrease, may be 'jumping' past the best model. Also try early stopping as the best model is unlikely to be the final one. 

Q. Load the MNIST data (introduced in Chapter 3), and split it into a training set, a validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation, and 10,000 for testing). 

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)

W0803 11:50:30.973899 139732972889856 deprecation.py:323] From <ipython-input-1-1b917109a13c>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0803 11:50:30.974681 139732972889856 deprecation.py:323] From /home/edd/.local/share/virtualenvs/ml_projects-PtpjDyvs/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0803 11:50:30.975424 139732972889856 deprecation.py:323] From /home/edd/.local/share/virtualenvs/ml_projects-PtpjDyvs/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.pytho

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz


W0803 11:50:31.214300 139732972889856 deprecation.py:323] From /home/edd/.local/share/virtualenvs/ml_projects-PtpjDyvs/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import numpy as np

In [3]:
X_train = mnist.train.images
y_train = mnist.train.labels

X_test = mnist.test.images
y_test = mnist.test.labels

X = np.concatenate([X_train, X_test])
y = np.concatenate([y_train, y_test])

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_to_split, X_val, y_to_split, y_val = train_test_split(X,
                                                        y,
                                                        train_size=55000,
                                                        random_state=42,
                                                        stratify=y) 

X_train, X_test, y_train, y_test = train_test_split(X_to_split,
                                                    y_to_split,
                                                    train_size=45000,
                                                    random_state=42,
                                                    stratify=y_to_split) 

Then train various classifiers, such as a Random Forest classifier, an Extra-Trees classifier, and an SVM.

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import SCORERS

In [7]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [8]:
np.random.seed(42)

In [9]:
rf_random_params = {"n_estimators": np.linspace(10,500,490).astype(int),
             "criterion":["gini", "entropy"],
             "max_depth": np.linspace(1,5,4),
             "max_features": np.linspace(0.1,1,100)}

In [10]:
rf_clf = RandomForestClassifier(random_state=42,verbose=3)

In [11]:
rf_rand = RandomizedSearchCV(rf_clf,
                             rf_random_params,
                             n_iter=250,
                             scoring="balanced_accuracy",
                             n_jobs=-1,
                             cv=3,
                             verbose=10,
                             random_state=42)

In [22]:
rf_rand.fit(X_train,y_train)

Fitting 3 folds for each of 250 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 21.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 21.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 26.9min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 45.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 57.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 71.6min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 83.4min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 86.8min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 93.2min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 100.9min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 116.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 132.1min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:

building tree 1 of 63


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


building tree 2 of 63


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


building tree 3 of 63
building tree 4 of 63
building tree 5 of 63
building tree 6 of 63
building tree 7 of 63
building tree 8 of 63
building tree 9 of 63
building tree 10 of 63
building tree 11 of 63
building tree 12 of 63
building tree 13 of 63
building tree 14 of 63
building tree 15 of 63
building tree 16 of 63
building tree 17 of 63
building tree 18 of 63
building tree 19 of 63
building tree 20 of 63
building tree 21 of 63
building tree 22 of 63
building tree 23 of 63
building tree 24 of 63
building tree 25 of 63
building tree 26 of 63
building tree 27 of 63
building tree 28 of 63
building tree 29 of 63
building tree 30 of 63
building tree 31 of 63
building tree 32 of 63
building tree 33 of 63
building tree 34 of 63
building tree 35 of 63
building tree 36 of 63
building tree 37 of 63
building tree 38 of 63
building tree 39 of 63
building tree 40 of 63
building tree 41 of 63
building tree 42 of 63
building tree 43 of 63
building tree 44 of 63
building tree 45 of 63
building tree 46 o

[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:   25.2s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [30]:
y_rf_preds = rf_rand.predict(X_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.4s finished


In [12]:
from sklearn.metrics import balanced_accuracy_score

In [27]:
balanced_accuracy_score(y_train, y_rf_preds)


0.8412291145370604

In [29]:
y_rf_test_preds = rf_rand.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.1s finished


In [31]:
balanced_accuracy_score(y_test, y_rf_test_preds)


0.8314416735561345

not bad

In [23]:
rf_rand.best_params_

{'n_estimators': 63,
 'max_features': 0.14545454545454545,
 'max_depth': 5.0,
 'criterion': 'gini'}

In [32]:
rf_params = {"n_estimators": [x for x in range(20,200,20)],
             "criterion":["entropy"],
             "max_depth": [3,4,5],
             "max_features": [x for x in np.arange(0.05,0.26,0.05)] }

In [33]:
rf_grid = GridSearchCV(rf_clf, rf_params, verbose=10, n_jobs=-1)

In [34]:
rf_grid.fit(X_train,y_train)

Fitting 3 folds for each of 135 candidates, totalling 405 fits


/home/edd/.local/share/virtualenvs/ml_projects-PtpjDyvs/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   58.9s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  6.9min
[Parallel(

building tree 1 of 180
building tree 2 of 180


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


building tree 3 of 180
building tree 4 of 180
building tree 5 of 180
building tree 6 of 180
building tree 7 of 180
building tree 8 of 180
building tree 9 of 180
building tree 10 of 180
building tree 11 of 180
building tree 12 of 180
building tree 13 of 180
building tree 14 of 180
building tree 15 of 180
building tree 16 of 180
building tree 17 of 180
building tree 18 of 180
building tree 19 of 180
building tree 20 of 180
building tree 21 of 180
building tree 22 of 180
building tree 23 of 180
building tree 24 of 180
building tree 25 of 180
building tree 26 of 180
building tree 27 of 180
building tree 28 of 180
building tree 29 of 180
building tree 30 of 180
building tree 31 of 180
building tree 32 of 180
building tree 33 of 180
building tree 34 of 180
building tree 35 of 180
building tree 36 of 180
building tree 37 of 180
building tree 38 of 180
building tree 39 of 180
building tree 40 of 180
building tree 41 of 180
building tree 42 of 180
building tree 43 of 180
building tree 44 of 180

[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:   36.4s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=3, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid

In [35]:
rf_grid.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 0.05,
 'n_estimators': 180}

In [36]:
y_rf_train_pred = rf_grid.predict(X_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:    1.2s finished


In [37]:
balanced_accuracy_score(y_train, y_rf_train_pred)

0.8540817639406963

In [38]:
y_rf_test_pred = rf_grid.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:    0.3s finished


In [39]:
balanced_accuracy_score(y_test, y_rf_test_pred)

0.8479108309726741

bit of an improvement there, lets try extra trees

In [13]:
from sklearn.ensemble import ExtraTreesClassifier

In [14]:
et_random_params = {"n_estimators": np.linspace(10,500,490).astype(int),
             "criterion":["gini", "entropy"],
             "max_depth": np.linspace(1,5,4),
             "max_features": np.linspace(0.1,1,100)}

In [15]:
et_clf = ExtraTreesClassifier(random_state=42,verbose=3)

In [16]:
et_rand = RandomizedSearchCV(et_clf,
                             et_random_params,
                             n_iter=250,
                             scoring="balanced_accuracy",
                             n_jobs=-1,
                             cv=3,
                             verbose=10,
                             random_state=42)

In [17]:
et_rand.fit(X_train,y_train)

Fitting 3 folds for each of 250 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 52.9min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 61.6min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 75.1min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 90.0min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 94.6min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 101.8min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 112.1min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 129.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 145.0min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed

building tree 1 of 215


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


building tree 2 of 215


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.4s remaining:    0.0s


building tree 3 of 215
building tree 4 of 215
building tree 5 of 215
building tree 6 of 215
building tree 7 of 215
building tree 8 of 215
building tree 9 of 215
building tree 10 of 215
building tree 11 of 215
building tree 12 of 215
building tree 13 of 215
building tree 14 of 215
building tree 15 of 215
building tree 16 of 215
building tree 17 of 215
building tree 18 of 215
building tree 19 of 215
building tree 20 of 215
building tree 21 of 215
building tree 22 of 215
building tree 23 of 215
building tree 24 of 215
building tree 25 of 215
building tree 26 of 215
building tree 27 of 215
building tree 28 of 215
building tree 29 of 215
building tree 30 of 215
building tree 31 of 215
building tree 32 of 215
building tree 33 of 215
building tree 34 of 215
building tree 35 of 215
building tree 36 of 215
building tree 37 of 215
building tree 38 of 215
building tree 39 of 215
building tree 40 of 215
building tree 41 of 215
building tree 42 of 215
building tree 43 of 215
building tree 44 of 215

[Parallel(n_jobs=1)]: Done 215 out of 215 | elapsed:  2.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=ExtraTreesClassifier(bootstrap=False,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators='warn',
                                                  n_jobs=None, oob_sco...
       439, 

In [18]:
et_rand.best_params_

{'n_estimators': 215,
 'max_features': 0.23636363636363636,
 'max_depth': 5.0,
 'criterion': 'gini'}

In [19]:
y_ef_train_pred = et_rand.predict(X_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 215 out of 215 | elapsed:    1.5s finished


In [20]:
balanced_accuracy_score(y_train,y_ef_train_pred)

0.8364638003352314

In [21]:
et_params = {"n_estimators": [x for x in range(100,300,20)],
             "criterion":["entropy"],
             "max_depth": [3,4,5],
             "max_features": [x for x in np.arange(0.15,0.35,0.05)] }

In [22]:
et_grid = GridSearchCV(et_clf, et_params, verbose=10, n_jobs=-1)